### Common Setup

In [1]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import json
import time
import urllib
import pandas as pd



In [2]:
# Environment details:

#master_host = '**** ADD HERE ****'
#master_host = 'dse-ac922h.cpolab.ibm.com'
#dli_rest_port = '9243'
#sc_rest_port = '8280'
protocol = 'http'

master_host = 'colonia05.platform'
#master_host = 'aienterprise4.aus.stglabs.ibm.com'
dli_rest_port = '9280'
sc_rest_port = '8280'


sc_rest_url =  protocol+'://'+master_host+':'+sc_rest_port+'/platform/rest/conductor/v1'
dl_rest_url = protocol+'://'+master_host+':'+dli_rest_port+'/platform/rest/deeplearning/v1'

print (sc_rest_url)
print (dl_rest_url)
# User login details
#wmla_user = '**** ADD HERE ****'
#wmla_pwd = '**** ADD HERE ****'

wmla_user = 'Admin'
wmla_pwd = 'Admin'

myauth = (wmla_user, wmla_pwd)

# Spark instance group details
#sig_name = '**** ADD HERE ****'
sigName = 'dli'

# REST call variables
commonHeaders = {'Accept': 'application/json'}


#startTuneUrl='%s://%s:%s/platform/rest/deeplearning/v1/hypersearch' % (protocol, master_host, dli_rest_port)
#sc_rest_url ='%s://%s:%d/platform/rest/conductor/v1' % (protocol, hostname, conductorport)

req = requests.Session()

http://colonia05.platform:8280/platform/rest/conductor/v1
http://colonia05.platform:9280/platform/rest/deeplearning/v1


### Health Check

Check if there is any existing hpo tasks and also verify the platform health

Rest API: **GET platform/rest/deeplearning/v1/hypersearch**
- Description: Get all the hpo task that the login user can access.
- OUTPUT: A list of hpo tasks and each one with the same format which can be found in the api doc.

In [ ]:
getTuneStatusUrl = dl_rest_url + '/hypersearch'
print ('getTuneStatusUrl: %s' %getTuneStatusUrl)
r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=myauth)

if not r.ok:
    print('check hpo task status failed: code=%s, %s'%(r.status_code, r.content))
else:
    if len(r.json()) == 0:
        print('There is no hpo task been created')
    for item in r.json():
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
        #print('Best:%s'%json.dumps(item.get('best'), sort_keys=True, indent=4))

### Launch a HPO task

#### Model file update to Run HPO

Model changes required from 2 perspective:
- Inject hyper-parameters for the sub-training during search
- Retrieve sub-training result metric

##### Model update part 1 - Inject hyper-parameters

The hyper-parameters will be supplied in a file called **config.json** with JSON format,located in the current working directory and can be read direcly as the following example snippet.

<pre>
hyper_params = json.loads(open("<b>config.json</b>").read())
learning_rate = float(hyper_params.get("<b>learning_rate</b>", "0.01"))
</pre>

After this, you can use these hyper-parameters during the model trainings. The **hyper-parameter name** and **value** type is defined through the search space part in body of REST call when launching a new hpo task.

##### Model update part 2 - Retrieve sub-training result metric

At the end of your training run, your code will need to create a file called **val_dict_list.json** with test metrics generated during training. These metrics will be used by the search algorithm to propose new sets of hyper-parameters. Please note that **val_dict_list.json** should be created under the result directory which can be retrieved through the environment variable **RESULT_DIR**.

<pre>
with open('{}/val_dict_list.json'.format(os.environ['<b>RESULT_DIR</b>']), 'w') as f:
    json.dump(test_metrics, f)
</pre>

The content of **val_dict_list.json** will be some thing as below, **step** is some thing optional meaning the training iteration or epochs, one of **loss** and **accuracy** can be the name of target metric to optimize, at least one metric need to be included here. The specific name of metric used to optimize (minimize or maximize) is defined in the body of REST call when launching a new hpo task. 

```
[
{‘step’: 1, ‘loss’:0.2487, ‘accuracy’: 0.4523},
{‘step’: 2, ‘loss’:0.1487, ‘accuracy’: 0.5523},
{‘step’: 3, ‘loss’:0.1087, ‘accuracy’: 0.6523},
…
]
```

#### Launch HPO task

REST API: **POST /platform/rest/deeplearning/v1/hypersearch**
- Description: Start a new HPO task
- Content-type: Multi-Form
- Multi-Form Data:
  - files: Model files tar package, ending with `.modelDir.tar`
  - form-filed: {‘data’: ‘String format of input parameters to start hpo task, let’s call it as **hpo_input** and show its specification later’}


#### Model file update to Run HPO

##### Package model files for training

Package the updated model files into a tar file ending with `.modelDir.tar`

In [3]:
import tarfile
import tempfile
import os
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
MODEL_DIR_SUFFIX = ".modelDir.tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
make_tarfile(tempFile, '/Users/Kelvin/Documents/CwSSolution/Enablement/IBMDeveloper/HPO/Dillards/tf-model')
files = {'file': open(tempFile, 'rb')}

##### Construct POST request data

**hpo_input** will be a Python dict or json format as below, convert to string when calling REST.

In [4]:
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': sigName,

            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            #
            'args': '--exec-start tensorflow --cs-datastore-meta type=fs --python-version 2.7\
                     --gpuPerWorker 1 --model-main convolutional_network.py --model-dir tf-model\
                     --trainImagesFile /dlidata/tf_mnist/train-images-idx3-ubyte.gz \
                     --trainLabelsFile /dlidata/tf_mnist/train-labels-idx1-ubyte.gz \
                     --testImagesFile /dlidata/tf_mnist/t10k-images-idx3-ubyte.gz \
                     --testLabelsFile /dlidata/tf_mnist/t10k-labels-idx1-ubyte.gz \
                     --debug-level debug'
                
        },
    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband
            'algorithm': 'Hyperband', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': -1,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 12,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 6, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
            # eta value to control the hyper-band search process
            'hyperbandEta': 3.0,
            #Additional parameters for the specified search algorithm and hyper-band get following too.
            'algoParams' : 
                [
                    {
                        # Name of the the maximum amount of resource that can be allocated to a single configuration
                        'name':'ResourceName', 
                        'value': 'epochs'
                    },
                    {
                        # Value of the the maximum amount of resource that can be allocated to a single configuration
                        'name':'ResourceValue',
                        'value':'81'
                    }
                    # This resource parameter will change during hyperband searching phase and its value will be put
                    # into the config.json for each sub-training too, here with the 'epoch' as key and value in the
                    # range 1-10.
                ]
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': [],
                 #step size to split the Range space. ONLY valid when type is Range
                 'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

##### Submit the Post request

Submit hpo task through the Post call and a hpo name/id as string format will get back.

In [5]:
startTuneUrl=dl_rest_url + '/hypersearch'
r = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, auth=myauth)

if r.ok:
    hpoName = r.json()
else: 
    print('\nModel submission failed with code={}, {}'. format(r.status_code, r.content))

In [7]:
import time

getHpoUrl = dl_rest_url +'/hypersearch/'+ hpoName

res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not res.ok:
    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
else:
    json_out=res.json()
    
    while json_out['state'] in ['SUBMITTED','RUNNING']:
        print('Hpo task %s state %s progress %s%%'%(hpoName, json_out['state'], json_out['progress']))
        time.sleep(90)
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
        json_out=res.json()
        
        experiments_length = len(json_out['experiments'])
       
        ####
        ## Query the list of 6 sub-training of current batch, as maxParalleJobNum=6
        ###      
        count=0
        Experiment = []
        while (count < experiments_length):
                appID = json_out['experiments'][count]['appId']
                #print ('appID: %s,' %appID )
                #print ('count: %d' %count)
                Experiment.insert(count, appID)
                count+=1
 
        ####
        ## Query the state of 6 sub-training of current batch
        ###
    
        count = 0
        while (count < len(Experiment)):
                r = requests.get(dl_rest_url+'/execs/'+Experiment[count], auth=myauth, headers=commonHeaders, verify=False).json()    
                if not res.ok:
                    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
                else:
                    print ('Experiement %s state: %s' %(Experiment[count], r['state']))
                count+=1
        
        #time.sleep(30)
        #print ('state:' + json_out['state'] )

        
print('Hpo task %s completes with state %s'%(hpoName, json_out['state']))
print(json.dumps(json_out, indent=4, sort_keys=True))
 


Hpo task Admin-hpo-1875218381975620 state RUNNING progress 0/12%
Experiement Admin-1875220138710388-1291894687 state: FINISHED
Experiement Admin-1875226015063937-337436081 state: FINISHED
Experiement Admin-1875231965093121-1383877797 state: RUNNING
Experiement Admin-1875237808422852-1610857758 state: RUNNING
Experiement Admin-1875243640156305-1151705668 state: RUNNING
Experiement Admin-1875249717056112-418184326 state: RUNNING
Hpo task Admin-hpo-1875218381975620 state RUNNING progress 0/12%
Experiement Admin-1875220138710388-1291894687 state: FINISHED
Experiement Admin-1875226015063937-337436081 state: FINISHED
Experiement Admin-1875231965093121-1383877797 state: FINISHED
Experiement Admin-1875237808422852-1610857758 state: FINISHED
Experiement Admin-1875243640156305-1151705668 state: RUNNING
Experiement Admin-1875249717056112-418184326 state: RUNNING
Hpo task Admin-hpo-1875218381975620 state RUNNING progress 0/12%
Experiement Admin-1875220138710388-1291894687 state: FINISHED
Experieme